In [333]:
#Tweet Retrieval Import

import GetOldTweets3 as got
import sys
import json

#Sentiment Analysis Imports
from textblob import TextBlob
import pandas as pd
import numpy as np

# Configuration

# Tweet Retrieval

In [334]:
class tweetFetcher:

### FUNCTIONS THAT HANDLE ACTOR LIST ###    
    
    #TODO needs to be changed to update actor_list based on how jennifer creates actor_lists in the other notebook

    def create_movie_actor_list_from_file(self, input_actor_file):
        
        with open(input_actor_file, 'r') as input_file:
            
            data = input_file.read()

            # parse file            
            self.movie_actor_list = json.loads(data)
        
    def set_movie_actor_list(self, provided_actor_list):
        self.movie_actor_list = provided_actor_list    
        
    def get_movie_actor_list(self):
        return self.movie_actor_list
    
### FUNCTIONS THAT QUERY DATA
    
    # Query all movies using internal movie_actor_list 
    # Store results of all tweets
    # Convert to JSON
    # Store in Output file
    
    def query_tweets_as_JSON(self, output_file, tweets_per_actor, movie_limit):
                    
        with open(output_file, 'w') as write_file:

            movie_tweet_object_list = []
            processed_query_results = []
            
            movie_counter = 0
            for movie in self.movie_actor_list:
                if movie_counter > movie_limit:
                    break            

                title = movie['title'] 
                release_date = movie['release_date']
                end_date = movie['end_date']
                actors = movie['actors']

                movie_dictionary = {'title': title, 'actors': []}

                for actor in actors:
                    
                    #Retrieve tweet objects for an actor and store them in a list
                    actor_tweet_object_list = self.query_tweets(actor, release_date, end_date, tweets_per_actor)
                    
                    actor_dictionary = {'name': actor, 'tweets': []}
                    
                    movie_dictionary['actors'].append(actor_dictionary)
                    

                    
                    #Convert each tweet_object into its data
                    for actor_tweet_object in actor_tweet_object_list:
                        actor_dictionary['tweets'].append(self.parse_tweet_object([actor_tweet_object]))

                

                processed_query_results.append(movie_dictionary)
                movie_counter += 1
                
                                        
                                        
            json.dump(processed_query_results, write_file)
    
    
    # Use the GOT3 module to get a tweet
    #Provide dates in "YYYY-MM-DD" Format
    def query_tweets(self, query, start_date = "2006-03-21", end_date = "2019-06-30", max_tweets = 1):
        tweetCriteria = got.manager.TweetCriteria().setQuerySearch(query)\
                                               .setSince(start_date)\
                                               .setUntil(end_date)\
                                               .setMaxTweets(max_tweets)
        
        queried_tweet_object_list = got.manager.TweetManager.getTweets(tweetCriteria)
        return queried_tweet_object_list

    
    
    # Given a list of tweet objects, return a list of dictionaries
    # where each dictionary looks like {date: strTweetDate, text: strText}
    def parse_tweet_object(self, tweet_object_list):
        
        tweet_data_list = []
            
        for tweet in tweet_object_list:
                            
            date = str(tweet.date)
            text = tweet.text
            tweet_data = {"date": date, "text": text}

            tweet_data_list.append(tweet_data)

                                
        return tweet_data_list

In [335]:
#Example of how to use TweetFetcher

def fetch_tweets(input_actor_file, output_tweet_file, tweets_per_actor, number_of_movies):
    tweet_fetcher = tweetFetcher()    
    
    tweet_fetcher.create_movie_actor_list_from_file(input_actor_file)
    
    tweet_fetcher.query_tweets_as_JSON(output_tweet_file, tweets_per_actor, number_of_movies)

    
fetch_tweets(input_actor_file = "intermediates/actors.json",
             output_tweet_file = "intermediates/data_file.json",
             tweets_per_actor = 4,
             number_of_movies = 5)

# Sentiment Analysis

In [336]:
class sentiment_analyzer:
    
    
    #Given a selection of raw text, return the sentiment of that text
    def get_text_sentiment(self, raw_text):
        sentiment = TextBlob(raw_text).sentiment
        return sentiment
    
    
    #TODO Fix this to use the new output from the new API
    def mass_tweet_analysis(self, archive_file):
        
        with open(archive_file) as json_file:  
            parsed = json.load(json_file)
            
            print(json.dumps(parsed, indent=4, sort_keys=True))
    
    
    ##Deprecated Method: Do not USE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1111111111111111111111111
    
    def update_tweet_sentiment(self, actor_tweet_dictionary):
    
        tweets_by_actor = actor_tweet_dictionary
        for actors in tweets_by_actor:
            for tweets in (tweets_by_actor[actors]):
                tweets.update(sentiment=TextBlob(tweets['content']).sentiment)
                print(tweets)
            
        return(tweets_by_actor)

In [337]:
sentiment_analyzer = sentiment_analyzer()

In [338]:
sentiment_analyzer.mass_tweet_analysis("intermediates/data_file.json")

[
    {
        "actors": [
            {
                "name": "Sam Worthington",
                "tweets": [
                    [
                        {
                            "date": "2010-01-09 23:38:55+00:00",
                            "text": "(Oddly, Awesome Wife and I confessed to quite fancying the Na'vi forms of Sam Worthington and Zoe Saldana respectively) #fb"
                        }
                    ],
                    [
                        {
                            "date": "2010-01-09 23:21:42+00:00",
                            "text": "Just watched Terminator Salvation. AGAIN. I love Sam Worthington!! Need to see Avatar : )"
                        }
                    ],
                    [
                        {
                            "date": "2010-01-09 22:36:34+00:00",
                            "text": "Watching Terminator Salvation. Sam Worthington here I come!"
                        }
                    ],
             